In [35]:
import cv2
import numpy as np
import pandas as pd

# Load the distorted dot array image
img = cv2.imread('C:\\Users\\WTA\\Desktop\\target\\0.46Xdot.bmp')

# Define the 3D coordinates of the dot array
col_pt_num = 15
row_pt_num = 36
objp = np.zeros((col_pt_num * row_pt_num, 3), np.float32)
objp[:, :2] = np.mgrid[0:row_pt_num, 0:col_pt_num].T.reshape(-1, 2)

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Find the corners of the dot array
ret, corners = cv2.findCirclesGrid(gray, (row_pt_num, col_pt_num), None)
distort_pt = np.asarray(corners).reshape(col_pt_num*row_pt_num,2)

# Calibrate the camera using the dot array corners and 3D coordinates
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera([objp], [corners], gray.shape[::-1], None, None)

# Undistort the image using the camera calibration parameters
undistorted_img = cv2.undistort(img, mtx, dist)
ret1, undistort_corners = cv2.findCirclesGrid(undistorted_img, (row_pt_num, col_pt_num), None)
center_points = np.asarray(undistort_corners).reshape(col_pt_num*row_pt_num,2)

# dataframe
df_distort = pd.DataFrame(distort_pt)
df_distort_row = pd.DataFrame(distort_pt.reshape(15,72))
df_undistort = pd.DataFrame(center_points)
df_undistort_row = pd.DataFrame(center_points.reshape(15,72))

#print(df_distort_row)

df_row1 = df_distort[[0,1]]
df_row2 = df_undistort[[0,1]]

for i in range(0,72,2):
    df_add1 = df_distort_row[[i,i+1]]
    df_add1.columns=[0,1]
    df_add2 = df_undistort_row[[i,i+1]]
    df_add2.columns=[0,1]
    df_distort680 = pd.concat([df_row1,df_add1],axis=0)
    df_undistort680 = pd.concat([df_row2,df_add2],axis=0)
df_distort680.columns = ['x','y']
df_undistort680.columns = ['x','y']
distort680 = np.asarray(df_distort680)
undistort680 = np.asarray(df_undistort680)

# origin
df_undistort.columns = ['x', 'y']

origin_x = df_undistort['x'].mean()
origin_y = df_undistort['y'].mean()


# Calculate the average distortion percentage for all points
h, w = gray.shape[::-1]
#print(len(corners))
#print(len(corners[0]))
num_points = 0
distortion_height = []
distortion_percentage = 0


In [36]:
print(pd.concat([df_distort_row[0],df_distort_row[1]],axis=1,ignore_index=True))

            0           1
0   37.117256   27.603130
1   37.218769   94.512871
2   37.326061  161.440689
3   37.416332  228.348984
4   37.480339  295.273254
5   37.575573  362.184448
6   37.668663  429.088379
7   37.761715  496.023895
8   37.854481  562.939941
9   37.930134  629.860596
10  38.025581  696.778931
11  38.124023  763.717773
12  38.204662  830.642578
13  38.289108  897.593384
14  38.382328  964.526428


In [37]:
new_rows=[]
for j in range(0, 72, 2):
    new_rows.append(pd.concat([df_distort_row[j], df_distort_row[j+1]],axis=1,ignore_index=True))

# combine the new columns into a new DataFrame
new_df = pd.concat(new_rows, axis=0)
distort680 = np.asarray(new_df)
new_rows1=[]
for j in range(0, 72, 2):
    new_rows1.append(pd.concat([df_undistort_row[j], df_undistort_row[j+1]],axis=1,ignore_index=True))

# combine the new columns into a new DataFrame
new_df1 = pd.concat(new_rows1, axis=0)
undistort680 = np.asarray(new_df1)

In [38]:
print(len(distort680))
print(undistort680)

540
[[  37.400066   27.619871]
 [  37.4915     94.53445 ]
 [  37.601173  161.44981 ]
 [  37.653778  228.34618 ]
 [  37.73328   295.26764 ]
 [  37.83639   362.18033 ]
 [  37.936543  429.0787  ]
 [  38.022964  495.99875 ]
 [  38.10309   562.91864 ]
 [  38.212673  629.8278  ]
 [  38.304832  696.7054  ]
 [  38.42189   763.6518  ]
 [  38.489147  830.57166 ]
 [  38.586796  897.49274 ]
 [  38.69737   964.39954 ]
 [ 104.28759    27.528831]
 [ 104.374664   94.44391 ]
 [ 104.45028   161.32166 ]
 [ 104.55133   228.24484 ]
 [ 104.63167   295.15833 ]
 [ 104.723114  362.06396 ]
 [ 104.84614   428.99973 ]
 [ 104.92035   495.88837 ]
 [ 105.0131    562.8111  ]
 [ 105.08619   629.72614 ]
 [ 105.202415  696.60236 ]
 [ 105.307304  763.537   ]
 [ 105.39634   830.44556 ]
 [ 105.47761   897.38306 ]
 [ 105.58219   964.2991  ]
 [ 171.17009    27.428934]
 [ 171.25479    94.32512 ]
 [ 171.33206   161.2326  ]
 [ 171.42656   228.1463  ]
 [ 171.54974   295.04004 ]
 [ 171.63248   361.95996 ]
 [ 171.70552   428.88223

In [39]:
np.set_printoptions(threshold=np.inf)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [40]:
num_points=0
col_pt_num=15
distortion_x=[]
distortion_height=[]
for i in range(len(distort680)):
    # Get the distorted and undistorted coordinates of the point
    pt_dist = distort680[i]
    pt_undist= undistort680[i]
    num_points += 1
    # Calculate the distortion percentage for this point
    distortion_percentage += np.linalg.norm(np.array(pt_dist) - np.array(pt_undist)) / np.linalg.norm(np.array(pt_undist)-np.array([[origin_x,origin_y]]))
        
    if num_points%col_pt_num == col_pt_num-1:
        print(pt_dist)
        distortion_percentage /= col_pt_num
        distortion_percentage *= 100
        distortion_height.append(distortion_percentage)
        distortion_percentage = 0
distortion_x = []
print(len(distortion_height)//2)
for i in range(len(distortion_height)//2):
    distortion_x.append((distortion_height[i]+distortion_height[row_pt_num-1-i])/2)
print(distortion_x[::-1])
distortion = np.mean(distortion_height)
print(f"Distortion percentage: {distortion:.5f}%")

[ 38.28911 897.5934 ]
[105.22272 897.46405]
[172.13545 897.3834 ]
[239.09015 897.27435]
[306.05106 897.1872 ]
[372.94424 897.0615 ]
[439.90927 896.9472 ]
[506.77747 896.87415]
[573.7379 896.7552]
[640.66034 896.64996]
[707.6047 896.5401]
[774.5148 896.4551]
[841.4348 896.3561]
[908.36743 896.26373]
[975.2823  896.17053]
[1042.1782   896.06305]
[1109.0929   896.00116]
[1176.0197   895.88776]
[1242.8804  895.829 ]
[1309.8339  895.7166]
[1376.7397  895.6098]
[1443.6143   895.52826]
[1510.5494  895.4353]
[1577.4581   895.34265]
[1644.3483   895.22656]
[1711.2444  895.1304]
[1778.1566  895.0108]
[1845.0421  894.9113]
[1911.9503  894.8064]
[1978.8434  894.7207]
[2045.7646  894.6365]
[2112.6284  894.5284]
[2179.5522  894.4301]
[2246.4749  894.3562]
[2313.341   894.2574]
[2380.2515  894.1837]
18
[0.001528037100797519, 0.0015688139440802238, 0.001866974234872032, 0.00295104258232944, 0.006129969127262787, 0.007558807155874092, 0.007693249050741239, 0.008564176981356772, 0.011147238255944103, 0.